# Chapter 4: Julia Sets
*Based on L. Keen*

## Summary

Julia sets arise from iteration of complex functions, particularly quadratic polynomials $f_c(z) = z^2 + c$. Key concepts:

- **Filled Julia set $K_c$**: Points whose orbits remain bounded
- **Julia set $J_c$**: Boundary of the filled Julia set (where dynamics are chaotic)
- **Connected vs. disconnected**: $J_c$ is connected iff $c$ is in the Mandelbrot set
- **Fatou set**: Complement of Julia set (where dynamics are stable)
- **Critical point**: $z = 0$ determines the entire dynamics

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numba import jit, prange
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown
%matplotlib inline
plt.style.use('dark_background')

## 4.1 Julia Set Visualization

For $f_c(z) = z^2 + c$, we color points by escape time.

In [ ]:
@jit(nopython=True, parallel=True)
def julia_set(c_real, c_imag, xmin, xmax, ymin, ymax, width, height, max_iter):
    """Compute Julia set escape times"""
    result = np.zeros((height, width))
    
    for i in prange(height):
        for j in range(width):
            zx = xmin + (xmax - xmin) * j / width
            zy = ymin + (ymax - ymin) * i / height
            
            n = 0
            while zx*zx + zy*zy < 4 and n < max_iter:
                zx, zy = zx*zx - zy*zy + c_real, 2*zx*zy + c_imag
                n += 1
            
            result[height - 1 - i, j] = n
    
    return result

def plot_julia(c_real=-0.7, c_imag=0.27, zoom=1.0, max_iter=200):
    """Plot Julia set with interactive controls"""
    half_width = 2.0 / zoom
    xmin, xmax = -half_width, half_width
    ymin, ymax = -half_width, half_width
    
    result = julia_set(c_real, c_imag, xmin, xmax, ymin, ymax, 800, 800, max_iter)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(result, extent=[xmin, xmax, ymin, ymax], 
                   cmap='magma', interpolation='bilinear')
    ax.set_xlabel('Re(z)')
    ax.set_ylabel('Im(z)')
    ax.set_title(f'Julia Set: c = {c_real} + {c_imag}i')
    plt.colorbar(im, label='Escape time')
    plt.show()

interact(plot_julia,
         c_real=FloatSlider(min=-1.5, max=1.5, step=0.01, value=-0.7, description='Re(c)'),
         c_imag=FloatSlider(min=-1.5, max=1.5, step=0.01, value=0.27, description='Im(c)'),
         zoom=FloatSlider(min=1, max=20, step=0.5, value=1, description='Zoom'),
         max_iter=IntSlider(min=50, max=500, step=50, value=200, description='Iterations'));

## 4.2 Famous Julia Sets

Explore some well-known Julia sets with interesting structures.

In [ ]:
famous_julia_sets = {
    'Dendrite (c = i)': (0.0, 1.0),
    'Douady Rabbit (c ≈ -0.123 + 0.745i)': (-0.123, 0.745),
    'San Marco (c ≈ -0.75)': (-0.75, 0.0),
    'Siegel Disk (c ≈ -0.391 - 0.587i)': (-0.391, -0.587),
    'Airplane (c ≈ -1.755)': (-1.755, 0.0),
    'Dragon (c ≈ -0.8 + 0.156i)': (-0.8, 0.156),
}

def plot_famous_julia(name):
    c_real, c_imag = famous_julia_sets[name]
    result = julia_set(c_real, c_imag, -2, 2, -2, 2, 800, 800, 300)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(result, extent=[-2, 2, -2, 2], cmap='twilight_shifted')
    ax.set_xlabel('Re(z)')
    ax.set_ylabel('Im(z)')
    ax.set_title(f'{name}')
    plt.show()

interact(plot_famous_julia,
         name=Dropdown(options=list(famous_julia_sets.keys()), 
                       value='Douady Rabbit (c ≈ -0.123 + 0.745i)',
                       description='Julia Set'));

## 4.3 Filled Julia Set vs Julia Set

The filled Julia set $K_c$ includes the interior; the Julia set $J_c$ is just the boundary.

In [ ]:
def plot_filled_vs_boundary(c_real=-0.1, c_imag=0.65):
    """Compare filled Julia set with its boundary"""
    result = julia_set(c_real, c_imag, -1.8, 1.8, -1.8, 1.8, 600, 600, 200)
    
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))
    
    # Filled Julia set (binary: escaped or not)
    filled = (result == 200).astype(float)
    axes[0].imshow(filled, extent=[-1.8, 1.8, -1.8, 1.8], cmap='binary')
    axes[0].set_title(f'Filled Julia Set $K_c$')
    axes[0].set_xlabel('Re(z)')
    axes[0].set_ylabel('Im(z)')
    
    # Julia set (boundary) - use gradient magnitude
    from scipy import ndimage
    gradient = ndimage.sobel(result)
    axes[1].imshow(np.abs(gradient), extent=[-1.8, 1.8, -1.8, 1.8], cmap='hot')
    axes[1].set_title(f'Julia Set $J_c$ (boundary)')
    axes[1].set_xlabel('Re(z)')
    axes[1].set_ylabel('Im(z)')
    
    fig.suptitle(f'c = {c_real} + {c_imag}i', fontsize=14)
    plt.tight_layout()
    plt.show()

interact(plot_filled_vs_boundary,
         c_real=FloatSlider(min=-1.5, max=0.5, step=0.05, value=-0.1, description='Re(c)'),
         c_imag=FloatSlider(min=-1.0, max=1.0, step=0.05, value=0.65, description='Im(c)'));

## 4.4 Orbit Behavior

Visualize how different starting points behave under iteration.

In [ ]:
def plot_orbit(c_real, c_imag, z_real, z_imag, n_iter):
    """Plot an orbit on top of the Julia set"""
    # Compute Julia set
    result = julia_set(c_real, c_imag, -2, 2, -2, 2, 500, 500, 200)
    
    # Compute orbit
    c = complex(c_real, c_imag)
    z = complex(z_real, z_imag)
    orbit_x, orbit_y = [z.real], [z.imag]
    
    for _ in range(n_iter):
        z = z**2 + c
        if abs(z) > 10:
            break
        orbit_x.append(z.real)
        orbit_y.append(z.imag)
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.imshow(result, extent=[-2, 2, -2, 2], cmap='magma', alpha=0.7)
    ax.plot(orbit_x, orbit_y, 'w-', lw=0.5, alpha=0.8)
    ax.plot(orbit_x, orbit_y, 'co', ms=3)
    ax.plot(orbit_x[0], orbit_y[0], 'go', ms=10, label='Start')
    ax.plot(orbit_x[-1], orbit_y[-1], 'ro', ms=10, label='End')
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    ax.legend()
    ax.set_title(f'Orbit of z₀ = {z_real} + {z_imag}i under f(z) = z² + c')
    plt.show()

interact(plot_orbit,
         c_real=FloatSlider(min=-1.5, max=0.5, step=0.05, value=-0.7, description='Re(c)'),
         c_imag=FloatSlider(min=-1.0, max=1.0, step=0.05, value=0.27, description='Im(c)'),
         z_real=FloatSlider(min=-1.5, max=1.5, step=0.1, value=0.5, description='Re(z₀)'),
         z_imag=FloatSlider(min=-1.5, max=1.5, step=0.1, value=0.5, description='Im(z₀)'),
         n_iter=IntSlider(min=10, max=200, step=10, value=50, description='Iterations'));

## Notes

- **Dichotomy theorem**: The Julia set is either connected or a Cantor set (totally disconnected)
- **Self-similarity**: Julia sets exhibit self-similarity at all scales
- **Periodic points**: Repelling periodic points are dense in the Julia set
- **Critical orbit**: The orbit of $z = 0$ determines whether $J_c$ is connected